In [1]:
import os
import json
import numpy as np

In [3]:
from cardiac.cadc_perfusion import gen_defect_extent_bull_eye_map, gen_defect_severity_bull_eye_map,gen_score_bull_eye
from cardiac.cadc_perfusion import cal_defect_extent, cal_defect_severity,cal_tdp, cal_categoric_scores

base_dir = r'./'
normal_limits_dir = os.path.join(base_dir, 'normal_limits')

In [4]:
def load_segments(base_dir=os.path.join(base_dir, 'segment_img')):
    segment17 = np.load(os.path.join(base_dir, 'segment17.npy')).astype(np.float32)
    segment_coronary_artery = np.load(os.path.join(base_dir,  'segment_coronary_artery.npy')).astype(np.float32)
    segment_myocardial_wall = np.load(os.path.join(base_dir,  'segment_myocardial_wall.npy')).astype(np.float32)

    return segment17, segment_coronary_artery, segment_myocardial_wall


def calculate_segment_categoric_scores(patient_be, limits_type='male_rest'):
    normal_limits = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_normal_limits.npy')).astype(np.float32)
    # normal_limits_sd = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_sd.npy')).astype(np.float32)

    segment17, segment_coronary_artery, segment_myocardial_wall = load_segments()

    categoric_score_bull_eye = gen_score_bull_eye(patient_be, normal_limits)
    categoric_scores_seg17 = cal_categoric_scores(categoric_score_bull_eye, segment17, False)
    categoric_scores_cor = cal_categoric_scores(categoric_score_bull_eye, segment_coronary_artery, False)
    categoric_scores_myo = cal_categoric_scores(categoric_score_bull_eye, segment_myocardial_wall, False)
    return {
        "Segment17Score":categoric_scores_seg17,
        "SegmentConoraryScore":categoric_scores_cor,
        "SegmentMyocardiumScore":categoric_scores_myo
    }

In [6]:


map_dict={
    1.0: 'LCX', 
    2.0: 'RCA',
    3.0: 'LAD',
    'total': 'TOT'
}

def calculate_conorary_score(patient_be, limits_type='male_rest', info=map_dict):
    normal_limits = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_normal_limits.npy')).astype(np.float32)
    normal_limits_sd = np.load(os.path.join(normal_limits_dir,  f'{limits_type}_sd.npy')).astype(np.float32)

    segment17, segment_coronary_artery, segment_myocardial_wall = load_segments()

    defect_extent_bull_eye = gen_defect_extent_bull_eye_map(patient_be, normal_limits)
    defect_extent_dict_cor = cal_defect_extent(defect_extent_bull_eye, segment_coronary_artery, False)

    score_bull_eye = gen_score_bull_eye(patient_be, normal_limits)
    tpd_dict_cor = cal_tdp(score_bull_eye, segment_coronary_artery, False)

    # print(defect_extent_dict_cor)
    res = {}
    for key, name in map_dict.items():
        res[name] = {
            'EXT': defect_extent_dict_cor.get(key),
            'TPD': tpd_dict_cor.get(key)
        }
    return res




import SimpleITK as sitk

commercial_software_gen_cad_bull_eye = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(base_dir, 'perfusion_demo_img', 'BullEye_cad'))).astype(np.float32)[1]

calculate_conorary_score(commercial_software_gen_cad_bull_eye)






{'LCX': {'EXT': 13.641188959660298, 'TPD': 0.6637437269091606},
 'RCA': {'EXT': 15.251373046049851, 'TPD': 0.9096135385334492},
 'LAD': {'EXT': 17.004747518342683, 'TPD': 1.6422359272837639},
 'TOT': {'EXT': 15.824423185143498, 'TPD': 1.2395801953971386}}

In [7]:
def calculate_IschemiaResult(patient_be, limits_type='male_rest', info=map_dict):
    res = calculate_conorary_score(patient_be, limits_type, info)
    return {
        'Area': 105,
        'Defect': res['TOT']['EXT']*105/100,
        'Extent': res['TOT']['EXT'],
        'TPD': res['TOT']['TPD']
    }
    # return 

calculate_IschemiaResult(commercial_software_gen_cad_bull_eye)

{'Area': 105,
 'Defect': 16.615644344400675,
 'Extent': 15.824423185143498,
 'TPD': 1.2395801953971386}